
# Blockchain — Notebook de Contrôle


## Table des matières
- [0. Objectifs, hypothèses, limites](#0-objectifs-hypothèses-limites)
- [1. Configuration Initiale](#1-configuration-initiale)
- [2. Introduction Théorique](#2-introduction-théorique)
- [Partie I — Blockchain Monoposte](#partie-i--blockchain-monoposte)
  - [1. Structure du Bloc](#11-structure-du-bloc)
  - [2. Proof of Work (Minage)](#12-proof-of-work-minage)
  - [3. Construction de la Blockchain](#13-construction-de-la-blockchain)
  - [4. Système de Transactions](#14-système-de-transactions)
  - [5. Validation et Sécurité](#15-validation-et-sécurité)
  - [6. API REST — Simulation](#16-api-rest--simulation)
  - [7. Statistiques & Analytics](#17-statistiques--analytics)
- [Partie II — Décentralisation (simulation)](#partie-ii--décentralisation-simulation)
  - [1. Architecture Multi-nœuds](#21-architecture-multi-nœuds)
  - [2. Communication Inter-nœuds](#22-communication-inter-nœuds)
  - [3. Synchronisation & résolution de conflits](#23-synchronisation--résolution-de-conflits)
  - [4. Scénarios avancés](#24-scénarios-avancés)
  - [5. Consensus distribué (vote simplifié)](#25-consensus-distribué-vote-simplifié)
  - [6. Métriques réseau](#26-métriques-réseau)
- [Section Finale](#section-finale)
- [Sources & Lectures complémentaires](#sources--lectures-complémentaires)

---

<a id="0-objectifs-hypothèses-limites"></a>
## **0. Objectifs, hypothèses, limites**

**Objectifs pédagogiques**
- Disséquer la **structure d’un bloc** et d’une **chaîne**.
- Implémenter une **preuve de travail (PoW)** et visualiser son coût.
- Gérer un **pool de transactions** et une **récompense de minage**.
- Simuler un **réseau de nœuds** et les mécanismes de synchronisation.

**Hypothèses simplificatrices**
- Pas de **signatures** cryptographiques des transactions (ECDSA), pas d’UTXO.
- **Validation distribuée** fortement simplifiée (aucun protocole de propagation réaliste).
- **Difficulté** fixée et faible pour la démonstration (exécution raisonnable).

**Limites**
- Non résistant à des attaques réseau réelles (Sybil, Eclipse, 51%).
- Pas de protection contre **double dépense** en environnement distribué réel.
- Le but est **l’apprentissage**, pas la sécurité opérationnelle.



<a id="1-configuration-initiale"></a>

## 1. Configuration Initiale


In [1]:

# Imports, constantes et utilitaires d'affichage
import hashlib, json, random, string, time
from dataclasses import dataclass, field, asdict
from datetime import datetime, timezone
from typing import Any, Dict, List, Optional

# Démonstration confortable (éviter des temps de minage trop longs)
DIFFICULTY = 3          # ajustez à 2 si votre machine est lente
MINING_REWARD = 50

random.seed(42)

# Couleurs console (optionnel)
COLORS = {"header": "\u001b[95m","ok": "\u001b[92m","warning": "\u001b[93m","info": "\u001b[96m","reset": "\u001b[0m"}
def highlight(text: str, color: str = "info") -> str:
    return f"{COLORS.get(color, COLORS['info'])}{text}{COLORS['reset']}"

def pretty_print_block(block_dict: Dict[str, Any]) -> None:
    print(highlight("Bloc", "header"))
    for key, value in block_dict.items():
        print(f"  - {key}: {value}")

def ascii_divider(title: str) -> None:
    line = "=" * 16
    print(f"{line} {title} {line}")

def utc_now_iso() -> str:
    """Horodatage ISO 8601 (RFC 3339) en UTC pour reproductibilité."""
    return datetime.now(timezone.utc).isoformat()



<a id="2-introduction-théorique"></a>

## 2. Introduction Théorique

La **blockchain** est un registre distribué immuable conservant un historique ordonné d'événements. Les blocs contiennent les transactions et sont liés cryptographiquement par leur **empreinte** (`hash`).

```
┌────────────┐     ┌────────────┐     ┌────────────┐
│  Bloc 0    │────▶│  Bloc 1    │────▶│  Bloc 2    │
│ hash=0000  │     │ hash=1ab2  │     │ hash=90cd  │
│ prev=None  │     │ prev=0000  │     │ prev=1ab2  │
└────────────┘     └────────────┘     └────────────┘
```

**Principes clés**
1. **Immutabilité** — Toute modification rompt les liens cryptographiques.
2. **Décentralisation** — Copie du registre répliquée entre plusieurs acteurs.
3. **Consensus** — Accord sur l’état valide (ici via **Proof of Work**).
4. **Transparence** — Vérifiabilité publique des transactions.



# Partie I — Blockchain Monoposte



<a id="11-structure-du-bloc"></a>

## 1.1 Structure du Bloc

Un bloc encapsule :
- `index` : position dans la chaîne
- `timestamp` : date ISO de création
- `transactions` : données utiles
- `previous_hash` : hash du bloc précédent
- `nonce` : compteur PoW
- `hash` : empreinte SHA-256 du bloc


In [2]:

@dataclass
class Block:
    """
    Bloc minimal pour démonstration pédagogique.
    """
    index: int
    timestamp: str
    transactions: List[Dict[str, Any]]
    previous_hash: str
    nonce: int = 0
    hash: str = field(init=False)

    def __post_init__(self) -> None:
        self.hash = self.calculate_hash()

    def calculate_hash(self) -> str:
        """
        Empreinte SHA-256 du bloc, calculée sur une sérialisation JSON canonique.
        Réf. SHA-256: FIPS 180-4 (NIST).
        """
        block_string = json.dumps(
            {
                "index": self.index,
                "timestamp": self.timestamp,
                "transactions": self.transactions,
                "previous_hash": self.previous_hash,
                "nonce": self.nonce,
            },
            sort_keys=True,
            separators=(",", ":"),
        ).encode()
        return hashlib.sha256(block_string).hexdigest()

    def mine_block(self, difficulty: int) -> Dict[str, Any]:
        """
        Preuve de travail: cherche un hash commençant par '0' * difficulty.
        Retourne statistiques (itérations, durée).
        """
        start = time.time()
        target = "0" * difficulty
        iterations = 0
        while not self.hash.startswith(target):
            self.nonce += 1
            iterations += 1
            self.hash = self.calculate_hash()
        duration = time.time() - start
        return {"iterations": iterations, "duration": duration}

    def to_dict(self) -> Dict[str, Any]:
        return asdict(self)

    def __repr__(self) -> str:
        return f"Block(index={self.index}, hash={self.hash[:10]}..., nonce={self.nonce})"


In [3]:

ascii_divider("Test du bloc")
block_example = Block(
    index=1,
    timestamp=utc_now_iso(),
    transactions=[{"sender": "alice", "receiver": "bob", "amount": 10}],
    previous_hash="0" * 64,
)
pretty_print_block(block_example.to_dict())

print("\nAprès modification d'une transaction :")
block_example.transactions[0]["amount"] = 15
print("Nouveau hash recalculé :", block_example.calculate_hash())
print("Ancien hash initial   :", block_example.hash)


================ Test du bloc ================
Bloc
  - index: 1
  - timestamp: 2025-11-09T19:59:53.287104+00:00
  - transactions: [{'sender': 'alice', 'receiver': 'bob', 'amount': 10}]
  - previous_hash: 0000000000000000000000000000000000000000000000000000000000000000
  - nonce: 0
  - hash: 4bcce2e0a3d8c2d7219b8e2ff5b7c9b1db833da1b1c2c4a70596d7f9a33df5ec

Après modification d'une transaction :
Nouveau hash recalculé : 1d52f89ee6d81a488d111a08331d6b21108e6f91b0ceba626282b584253f8b69
Ancien hash initial   : 4bcce2e0a3d8c2d7219b8e2ff5b7c9b1db833da1b1c2c4a70596d7f9a33df5ec



<a id="12-proof-of-work-minage"></a>

## 1.2 Proof of Work (Minage)

La preuve de travail consiste à trouver un **nonce** tel que le hash du bloc commence par un nombre donné de zéros (`difficulty`). Ce mécanisme régule l’ajout des blocs et sécurise la chaîne.


In [4]:

for difficulty in (2, 3):
    ascii_divider(f"Minage difficulté {difficulty}")
    block = Block(
        index=1,
        timestamp=utc_now_iso(),
        transactions=[{"sender": "system", "receiver": "miner", "amount": 1}],
        previous_hash="0" * 64,
    )
    stats = block.mine_block(difficulty)
    print("Hash trouvé :", block.hash)
    print("Itérations  :", stats["iterations"], "| Temps (s):", round(stats["duration"], 4))


================ Minage difficulté 2 ================
Hash trouvé : 005472ead0475e2e52cf86eba238fdd51ef3d630b46e6be9aa71b80150c6a943
Itérations  : 232 | Temps (s): 0.0021
================ Minage difficulté 3 ================
Hash trouvé : 000151002d4b828047d0b137f8e545028d0aed7a28c0b6edbe90cef22cfc3052
Itérations  : 3192 | Temps (s): 0.0171



<a id="13-construction-de-la-blockchain"></a>

## 1.3 Construction de la Blockchain

- Le **bloc genesis** démarre la chaîne.
- Chaque nouveau bloc est **miné** avant d'être ajouté.
- `is_chain_valid` recalcule les hash et vérifie la preuve de travail.


In [5]:

class Blockchain:
    """
    Chaîne pédagogique avec mémoire de transactions en attente et récompense de minage.
    """
    def __init__(self, difficulty: int = DIFFICULTY) -> None:
        self.difficulty = difficulty
        self.chain: List[Block] = [self.create_genesis_block()]
        self.pending_transactions: List[Dict[str, Any]] = []
        self.mining_reward = MINING_REWARD

    def create_genesis_block(self) -> Block:
        return Block(
            index=0,
            timestamp=utc_now_iso(),
            transactions=[{"sender": "system", "receiver": "genesis", "amount": 0}],
            previous_hash="0" * 64,
        )

    def get_latest_block(self) -> Block:
        return self.chain[-1]

    def add_block(self, block: Block) -> None:
        block.previous_hash = self.get_latest_block().hash
        block.hash = block.calculate_hash()
        block.mine_block(self.difficulty)
        self.chain.append(block)

    def add_transaction(self, transaction: Dict[str, Any]) -> None:
        self.pending_transactions.append(transaction)

    def mine_pending_transactions(self, miner_address: str) -> Block:
        reward_tx = {"sender": "system", "receiver": miner_address, "amount": self.mining_reward}
        block_transactions = self.pending_transactions + [reward_tx]
        block = Block(
            index=len(self.chain),
            timestamp=utc_now_iso(),
            transactions=block_transactions,
            previous_hash=self.get_latest_block().hash,
        )
        stats = block.mine_block(self.difficulty)
        self.chain.append(block)
        self.pending_transactions = []
        block.metadata = stats  # annotation légère pour analytics
        return block

    def is_chain_valid(self) -> bool:
        target = "0" * self.difficulty
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i - 1]
            if current.hash != current.calculate_hash():
                return False
            if current.previous_hash != previous.hash:
                return False
            if not current.hash.startswith(target):
                return False
        return True

    def get_balance(self, address: str) -> int:
        balance = 0
        for block in self.chain:
            for tx in block.transactions:
                if tx["sender"] == address:
                    balance -= tx["amount"]
                if tx["receiver"] == address:
                    balance += tx["amount"]
        return balance


In [6]:

blockchain = Blockchain(difficulty=3)

for sender in ("alice", "bob", "charlie"):
    tx = {"sender": sender, "receiver": "market", "amount": random.randint(1, 5)}
    blockchain.add_transaction(tx)

mined_block = blockchain.mine_pending_transactions("miner_1")
print("Bloc miné :", mined_block)
print("Validité de la chaîne :", blockchain.is_chain_valid())
print("Balances :")
for person in ("alice", "bob", "charlie", "miner_1", "market"):
    print(f" - {person}: {blockchain.get_balance(person)}")


Bloc miné : Block(index=1, hash=000a3c0b3a..., nonce=989)
Validité de la chaîne : True
Balances :
 - alice: -1
 - bob: -1
 - charlie: -3
 - miner_1: 50
 - market: 5



<a id="14-système-de-transactions"></a>

## 1.4 Système de Transactions


In [7]:

@dataclass
class Transaction:
    sender: str
    receiver: str
    amount: int

    def to_dict(self) -> Dict[str, Any]:
        return asdict(self)

    def is_valid(self) -> bool:
        return (
            isinstance(self.sender, str)
            and isinstance(self.receiver, str)
            and isinstance(self.amount, (int, float))
            and self.amount > 0
        )


In [8]:

market_chain = Blockchain(difficulty=2)

transactions = [
    Transaction("alice", "bob", 5),
    Transaction("bob", "charlie", 2),
    Transaction("alice", "charlie", 1),
]

for tx in transactions:
    if tx.is_valid():
        market_chain.add_transaction(tx.to_dict())

market_chain.mine_pending_transactions("miner_2")
print("Balances après minage :")
for person in ("alice", "bob", "charlie", "miner_2"):
    print(f" - {person}: {market_chain.get_balance(person)}")

print("\nTableau des balances")
print("| Adresse   | Solde |")
print("|-----------|-------|")
for person in ("alice", "bob", "charlie", "miner_2"):
    print(f"| {person:<9}| {market_chain.get_balance(person):>5} |")


Balances après minage :
 - alice: -6
 - bob: 3
 - charlie: 3
 - miner_2: 50

Tableau des balances
| Adresse   | Solde |
|-----------|-------|
| alice    |    -6 |
| bob      |     3 |
| charlie  |     3 |
| miner_2  |    50 |



<a id="15-validation-et-sécurité"></a>

## 1.5 Validation et Sécurité


In [9]:

def run_validation_tests(chain: Blockchain) -> Dict[str, bool]:
    scenarios = {}
    scenarios["intact"] = chain.is_chain_valid()

    tampered_chain = Blockchain(difficulty=chain.difficulty)
    tampered_chain.chain = [block for block in chain.chain]
    tampered_chain.chain[1].transactions[0]["amount"] += 99
    scenarios["tampered_data"] = tampered_chain.is_chain_valid()

    forged_chain = Blockchain(difficulty=chain.difficulty)
    forged_chain.chain = [block for block in chain.chain]
    forged_chain.chain[1].previous_hash = "12345"
    scenarios["tampered_link"] = forged_chain.is_chain_valid()

    return scenarios

validation_results = run_validation_tests(blockchain)
for name, result in validation_results.items():
    print(f"Scenario {name}: {result}")


Scenario intact: True
Scenario tampered_data: False
Scenario tampered_link: False



<a id="16-api-rest--simulation"></a>

## 1.6 API REST — FastAPI

Exposition de la blockchain via une API REST construite avec FastAPI. Cette section fournit des routes HTTP complètes pouvant être servies via `uvicorn`.

**Routes exposées**
- `GET /health` : vérifie que le service répond.
- `GET /chain` : retourne la chaîne complète et sa longueur.
- `POST /transactions` : enfile une transaction en attente de minage.
- `POST /mine` : mine les transactions en cours avec l'adresse du mineur fournie.
- `GET /balance/{address}` : calcule le solde courant d'une adresse.

> Pour lancer le serveur en dehors du notebook, créer un script minimal qui importe l'objet `app` puis exécuter `uvicorn <module>:app --reload`. Dans le notebook, on peut aussi démarrer `uvicorn.run(app, host="0.0.0.0", port=8000)` dans une cellule dédiée.


In [11]:

from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.testclient import TestClient
from pydantic import BaseModel

app = FastAPI(title="Educational Blockchain API", version="0.1.0")
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


class TransactionPayload(BaseModel):
    sender: str
    receiver: str
    amount: int


class MinePayload(BaseModel):
    miner_address: str


@app.get("/health")
def health_check() -> Dict[str, str]:
    return {"status": "ok"}


@app.get("/chain")
def read_chain() -> Dict[str, Any]:
    return {"length": len(blockchain.chain), "chain": [block.to_dict() for block in blockchain.chain]}


@app.post("/transactions", status_code=201)
def create_transaction(payload: TransactionPayload) -> Dict[str, Any]:
    tx = Transaction(payload.sender, payload.receiver, payload.amount)
    if not tx.is_valid():
        raise HTTPException(status_code=400, detail="Invalid transaction payload")
    blockchain.add_transaction(tx.to_dict())
    return {"status": "queued", "queue_length": len(blockchain.pending_transactions)}


@app.post("/mine")
def mine_block(payload: MinePayload) -> Dict[str, Any]:
    if not payload.miner_address:
        raise HTTPException(status_code=400, detail="Missing miner address")
    block = blockchain.mine_pending_transactions(payload.miner_address)
    return {
        "status": "success",
        "block_hash": block.hash,
        "transactions": len(block.transactions),
        "metadata": getattr(block, "metadata", {}),
    }


@app.get("/balance/{address}")
def read_balance(address: str) -> Dict[str, Any]:
    return {"address": address, "balance": blockchain.get_balance(address)}


In [12]:

client = TestClient(app)

print("GET /health ->", client.get("/health").json())
print("POST /transactions ->", client.post("/transactions", json={"sender": "dave", "receiver": "eve", "amount": 3}).json())
print("POST /mine ->", client.post("/mine", json={"miner_address": "miner_api"}).json())
print("GET /balance/dave ->", client.get("/balance/dave").json())
chain_preview = client.get("/chain").json()
print("GET /chain ->", chain_preview["length"], "blocs | premier hash =", chain_preview["chain"][0]["hash"])


GET /health -> {'status': 'ok'}
POST /transactions -> {'status': 'queued', 'queue_length': 1}
POST /mine -> {'status': 'success', 'block_hash': '0005cfab0d412c26d7bf4a2beccc3aea013d66dc9af0d6e858ed3022e0f5d5d9', 'transactions': 2, 'metadata': {'iterations': 7203, 'duration': 0.04699969291687012}}
GET /balance/dave -> {'address': 'dave', 'balance': -3}
GET /chain -> 3 blocs | premier hash = 8659027a0b01c47f8be42d55b786486031cb9706677c274b54adebe4a7651ea5



<a id="17-statistiques--analytics"></a>

## 1.7 Statistiques & Analytics


In [13]:

def get_blockchain_stats(chain: Blockchain) -> Dict[str, Any]:
    total_transactions = sum(len(block.transactions) for block in chain.chain)
    addresses: Dict[str, int] = {}
    total_iterations = 0
    mined_blocks = 0
    for block in chain.chain[1:]:
        for tx in block.transactions:
            addresses[tx["sender"]] = addresses.get(tx["sender"], 0) + 1
            addresses[tx["receiver"]] = addresses.get(tx["receiver"], 0) + 1
        metadata = getattr(block, "metadata", None)
        if metadata:
            total_iterations += metadata.get("iterations", 0)
            mined_blocks += 1
    return {
        "total_blocks": len(chain.chain),
        "total_transactions": total_transactions,
        "most_active": sorted(addresses.items(), key=lambda item: item[1], reverse=True)[:3],
        "avg_iterations": total_iterations / mined_blocks if mined_blocks else 0,
    }

stats = get_blockchain_stats(blockchain)
print(json.dumps(stats, indent=2))


{
  "total_blocks": 3,
  "total_transactions": 7,
  "most_active": [
    [
      "market",
      3
    ],
    [
      "system",
      2
    ],
    [
      "alice",
      1
    ]
  ],
  "avg_iterations": 4096.0
}



# Partie II — Décentralisation (simulation)



<a id="21-architecture-multi-nœuds"></a>

## 2.1 Architecture Multi-nœuds


In [14]:

class Node:
    def __init__(self, node_id: str) -> None:
        self.node_id = node_id
        self.blockchain = Blockchain(difficulty=2)
        self.peers: List["Node"] = []

    def register_peer(self, peer: "Node") -> None:
        if peer not in self.peers:
            self.peers.append(peer)

    def get_peers(self) -> List[str]:
        return [peer.node_id for peer in self.peers]

nodes = [Node(f"node_{i}") for i in range(3)]
for node in nodes:
    for peer in nodes:
        if peer is not node:
            node.register_peer(peer)

for node in nodes:
    print(node.node_id, "peers ->", node.get_peers())


node_0 peers -> ['node_1', 'node_2']
node_1 peers -> ['node_0', 'node_2']
node_2 peers -> ['node_0', 'node_1']



<a id="22-communication-inter-nœuds"></a>

## 2.2 Communication Inter-nœuds


In [15]:

class Network:
    def __init__(self, nodes: List[Node]):
        self.nodes = nodes

    def broadcast_block(self, sender: Node, block: Block) -> None:
        for node in self.nodes:
            if node is not sender:
                node.blockchain.chain.append(block)

    def synchronize_transaction(self, transaction: Transaction) -> None:
        for node in self.nodes:
            node.blockchain.add_transaction(transaction.to_dict())

network = Network(nodes)

broadcast_block = Block(
    index=1,
    timestamp=utc_now_iso(),
    transactions=[{"sender": "network", "receiver": "everyone", "amount": 1}],

    previous_hash=nodes[0].blockchain.get_latest_block().hash,
)
broadcast_block.mine_block(2)
network.broadcast_block(nodes[0], broadcast_block)

for node in nodes:
    print(node.node_id, "longueur de chaîne ->", len(node.blockchain.chain))


node_0 longueur de chaîne -> 1
node_1 longueur de chaîne -> 2
node_2 longueur de chaîne -> 2



<a id="23-synchronisation--résolution-de-conflits"></a>

## 2.3 Synchronisation & résolution de conflits


In [16]:

def resolve_conflicts(node: Node) -> bool:
    max_length = len(node.blockchain.chain)
    new_chain = None
    for peer in node.peers:
        if len(peer.blockchain.chain) > max_length and peer.blockchain.is_chain_valid():
            max_length = len(peer.blockchain.chain)
            new_chain = peer.blockchain.chain
    if new_chain:
        node.blockchain.chain = [block for block in new_chain]
        return True
    return False

nodes[1].blockchain.chain = nodes[1].blockchain.chain[:-1]
conflict_resolved = resolve_conflicts(nodes[1])
print("Conflit résolu ?", conflict_resolved)
print("Taille chaîne node_1 :", len(nodes[1].blockchain.chain))


Conflit résolu ? False
Taille chaîne node_1 : 1



<a id="24-scénarios-avancés"></a>

## 2.4 Scénarios avancés


In [17]:

def scenario_partition(network: Network) -> None:
    print("Partition réseau : node_2 isolé")
    isolated = network.nodes[-1]
    isolated.peers = []

    tx = Transaction("isolated", "market", 5)
    network.synchronize_transaction(tx)
    isolated.blockchain.mine_pending_transactions("isolated_miner")
    print("Chaîne isolée :", len(isolated.blockchain.chain))

def scenario_malicious(network: Network) -> None:
    print("Noeud malveillant tente d'injecter un faux bloc")
    malicious = network.nodes[0]
    fake_block = Block(
        index=99,
        timestamp=utc_now_iso(),
        transactions=[{"sender": "attacker", "receiver": "attacker", "amount": 1000}],
        previous_hash="123",
    )
    network.broadcast_block(malicious, fake_block)
    for node in network.nodes[1:]:
        print(node.node_id, "validité", node.blockchain.is_chain_valid())

def scenario_double_spend(network: Network) -> None:
    print("Tentative de double dépense")
    tx1 = Transaction("alice", "vendor", 10)
    tx2 = Transaction("alice", "vendor", 10)
    network.synchronize_transaction(tx1)
    network.synchronize_transaction(tx2)
    mined = network.nodes[1].blockchain.mine_pending_transactions("miner_network")
    print("Bloc miné double spend?",
          any(t["sender"] == "alice" and t["receiver"] == "vendor" and t["amount"] == 10
              for t in mined.transactions))

scenario_partition(network)
scenario_malicious(network)
scenario_double_spend(network)


Partition réseau : node_2 isolé
Chaîne isolée : 3
Noeud malveillant tente d'injecter un faux bloc
node_1 validité False
node_2 validité False
Tentative de double dépense
Bloc miné double spend? True



<a id="25-consensus-distribué-vote-simplifié"></a>

## 2.5 Consensus distribué (vote simplifié)


In [18]:

def vote_on_chain(nodes: List[Node]) -> Dict[str, int]:
    votes: Dict[str, int] = {}
    for node in nodes:
        chain_length = len(node.blockchain.chain)
        votes[str(chain_length)] = votes.get(str(chain_length), 0) + 1
    return votes

votes = vote_on_chain(nodes)
print("Votes par longueur de chaîne :", votes)
winning_length = max(votes, key=votes.get)
print("Consensus -> longueur", winning_length)


Votes par longueur de chaîne : {'1': 1, '3': 1, '4': 1}
Consensus -> longueur 1



<a id="26-métriques-réseau"></a>

## 2.6 Métriques réseau


In [19]:

def get_network_metrics(network: Network) -> Dict[str, Any]:
    return {
        "nodes": len(network.nodes),
        "avg_chain_length": sum(len(node.blockchain.chain) for node in network.nodes) / len(network.nodes),
        "pending_transactions": sum(len(node.blockchain.pending_transactions) for node in network.nodes),
    }

print(get_network_metrics(network))


{'nodes': 3, 'avg_chain_length': 2.6666666666666665, 'pending_transactions': 5}



# Section Finale


In [20]:

def run_full_test_suite() -> Dict[str, Any]:
    report: Dict[str, Any] = {}
    report["blockchain_valid"] = blockchain.is_chain_valid()
    report["stats"] = stats
    report["network_metrics"] = get_network_metrics(network)
    report["votes"] = votes
    return report

report = run_full_test_suite()
print(json.dumps(report, indent=2))

print(highlight("Playground interactif"))
name = "user_" + ''.join(random.choices(string.ascii_lowercase, k=4))
blockchain.add_transaction({"sender": name, "receiver": "market", "amount": random.randint(1, 5)})
mined = blockchain.mine_pending_transactions("play_miner")
print("Nouveau bloc miné :", mined)
print("Solde play_miner  :", blockchain.get_balance("play_miner"))


{
  "blockchain_valid": false,
  "stats": {
    "total_blocks": 3,
    "total_transactions": 7,
    "most_active": [
      [
        "market",
        3
      ],
      [
        "system",
        2
      ],
      [
        "alice",
        1
      ]
    ],
    "avg_iterations": 4096.0
  },
  "network_metrics": {
    "nodes": 3,
    "avg_chain_length": 2.6666666666666665,
    "pending_transactions": 5
  },
  "votes": {
    "1": 1,
    "3": 1,
    "4": 1
  }
}
Playground interactif
Nouveau bloc miné : Block(index=3, hash=0001bf6a85..., nonce=2452)
Solde play_miner  : 50
